In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

In [2]:
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Preprocessing

In [3]:
dataset = dataset.drop(["RowNumber","CustomerId","Surname"],axis=1)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset[["Gender","Geography"]]

,Gender,Geography
0,Female,France
1,Female,Spain
2,Female,France
3,Female,France
4,Female,Spain
...,...,...
9995,Male,France
9996,Male,France
9997,Female,France
9998,Male,Germany


In [5]:
### Label Encoding Gender
encoder_gen = LabelEncoder()
dataset['Gender'] = encoder_gen.fit_transform(dataset["Gender"])
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
### OneHot Encoder Geography
encoder_geo = OneHotEncoder()
encoded_geo = encoder_geo.fit_transform(dataset[["Geography"]]).toarray()
encoded_geo_df = pd.DataFrame(encoded_geo,columns = encoder_geo.get_feature_names_out(["Geography"]))
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
dataset = pd.concat([dataset.drop(['Geography'],axis=1),encoded_geo_df],axis=1)
dataset

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [8]:
### Train Test Split

x = dataset.drop("EstimatedSalary",axis=1)
y = dataset["EstimatedSalary"]

In [33]:
### Saving the encoders

with open("label_encoder_gen.pkl","wb") as file:
    pickle.dump(encoder_gen,file)

with open("one_hot_enc_geo.pkl","wb") as file:
    pickle.dump(encoder_geo,file)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [12]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [34]:
with open("regression_scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

### Training ANN for Regression

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
(x_train.shape[1],)

(12,)

In [18]:
### Build the model

model = Sequential([
    Dense(64, activation = "relu",input_shape = (x_train.shape[1],)),
    Dense(32,activation = "relu"),
    Dense(1)   ### default activation : Linear for regression
])

c:\Users\ViditAgarwal\Desktop\AI ML learned code\Deep Learning\ANN Churn Modelling\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.compile(optimizer='adam',loss = "mean_absolute_error",metrics=["mae"])

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

###Setting up TensorBoard
log_dir = "regression_logs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)

In [25]:
### Setting up early stopping
early_stopping_callback = EarlyStopping(monitor = "val_loss",patience=10,restore_best_weights=True)

In [26]:
history  = model.fit(
    x_train,y_train,
    validation_data = (x_test,y_test),
    epochs=100,
    callbacks = [early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 99606.4453 - mae: 99606.4453 - val_loss: 99784.1953 - val_mae: 99784.1953
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 98980.9062 - mae: 98980.9062 - val_loss: 98399.8750 - val_mae: 98399.8750
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 97886.9922 - mae: 97886.9922 - val_loss: 94849.5156 - val_mae: 94849.5156
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 93081.9219 - mae: 93081.9219 - val_loss: 88886.0312 - val_mae: 88886.0312
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 86353.5469 - mae: 86353.5469 - val_loss: 80949.3359 - val_mae: 80949.3359
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 77463.1406 - mae: 77463.1406 - val_loss: 72053.3203 - val_mae: 72053.3203
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 69408.1406 - mae: 69408.1406 - val_loss: 63998.1836 - val_mae: 63998.1836
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

In [32]:
model.save("regression_model.h5")

In [27]:
%load_ext tensorboard

In [30]:
%tensorboard --logdir regression_logs/fit20250223-015014

Reusing TensorBoard on port 6006 (pid 15472), started 0:00:08 ago. (Use '!kill 15472' to kill it.)

In [31]:
### Evaluate the model on test data

test_loss,test_mae = model.evaluate(x_test,y_test)

print("Test MAE:{}".format(test_mae))
print("Test Loss:{}".format(test_loss))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50120.5703 - mae: 50120.5703
Test MAE:50046.34765625
Test Loss:50046.34765625


#### Load the model

- Go to new ipynb file -- check there how it is done